# Data Cleaning & Exploratory Data Analysis
### Some Potential Ideas:
1. Sentiment Analysis with `review` column
2. Using   `recipe` column and feature engineering (length of `recipe`, TF-IDF, ...) to predict `ratings`

## Four Pillars of Data Cleaning
When loading in a dataset, to clean the data – that is, to prepare it for further analysis (machine learning, all kinds of stuff) – we will:
1. Perform data quality checks. Are they useful?

2. Identify and handle missing values.

3. Perform transformations, including converting time series data to timestamps.

4. Modify structure as necessary.

In [92]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [307]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
pd.options.plotting.backend = 'plotly'
from itertools import chain

from utils.eda import *
from utils.dsc80_utils import *
from utils.graph import *
from utils.missing_m import *

In [94]:
interactions = pd.read_csv('food_data/RAW_interactions.csv')
recipes = pd.read_csv('food_data/RAW_recipes.csv')

In [333]:
interactions.shape

(731927, 5)

In [331]:
recipes.shape

(83782, 12)

## Merge:
1. Left merge the recipes and interactions datasets together.

2. In the merged dataset, fill all ratings of 0 with np.nan. (Think about why this is a reasonable step, and include your justification in your website.)

3. Find the average rating per recipe, as a Series.

4. Add this Series containing the average rating per recipe back to the recipes dataset however you’d like (e.g., by merging). Use the resulting dataset for all of your analysis. (For the purposes of Project 4, the 'review' column in the interactions dataset doesn’t have much use.)

In [97]:
step0 = recipes.merge(interactions, how='left', left_on='id', right_on='recipe_id')
# merge on ones that are all in recipes
step0.shape

(234429, 17)

In [98]:
merged = step0.pipe(initial)

In [99]:
display(merged)

,name,id,minutes,contributor_id,...,date,rating,review,avg_rating
0,1 brownies in the world best ever,333281,40,985201,...,2008-11-19,4.0,"These were pretty good, but took forever to ba...",4.0
1,1 in canada chocolate chip cookies,453467,45,1848091,...,2012-01-26,5.0,Originally I was gonna cut the recipe in half ...,5.0
2,412 broccoli casserole,306168,40,50969,...,2008-12-31,5.0,This was one of the best broccoli casseroles t...,5.0
...,...,...,...,...,...,...,...,...,...
234426,cookies by design sugar shortbread cookies,298509,20,506822,...,2008-06-19,1.0,This recipe tastes nothing like the Cookies by...,2.0
234427,cookies by design sugar shortbread cookies,298509,20,506822,...,2010-02-08,5.0,"yummy cookies, i love this recipe me and my sm...",2.0
234428,cookies by design sugar shortbread cookies,298509,20,506822,...,2014-11-01,0.0,I work at a Cookies By Design and can say this...,2.0


## Data Cleaning & Transformation:
**Analysis**:
1. Some columns, like `nutrition`, contain values that look like lists, but are actually strings that look like lists. You may want to turn the strings into actual lists, or create columns for every unique value in those lists.
    - For instance, per the data dictionary, each value in the 'nutrition' column contains information in the form: "[calories (#), total fat (PDV), sugar (PDV), sodium (PDV), protein (PDV), saturated fat (PDV), and carbohydrates (PDV)]"; you could create individual columns in your dataset titled 'calories', 'total fat', etc.
2. Convert to list for `steps`, `ingredients`, and `tags`
3. Convert `date` and `submitted` to Timestamp object and rename as `review_date` and `recipe_date`
4. Convert Types
5. Drop same `id` (same with `recipe_id`)

**Report**:
Describe, in detail, the data cleaning steps you took and how they affected your analyses. The steps should be explained in reference to the data generating process. Show the head of your cleaned DataFrame (see Part 2: Report for instructions).

In [299]:
cleaned = merged.pipe(transform_df)

**type logic**
1. `String`: [name, contributor_id, user_id, recipe_id, ]
    - quantitative or qualitative, but cannot perform mathamatical operations (**quntitative discrete**)
    - `name` is the name of recipe
    - `contributor_id` is the author id of the recipe _(shape=10609)_
    - `recipe_id` is the id of teh recipe _(shape=45686)_
         - `id` from the original dataframe also is the id of the recipe, dropped after merging
    - `user_id` is the id of the reviewer _(shape=13751)_
2. `List`: [tags, steps, description, ingredients, review]
    - qualitative, no mathamatical operation (**qualitative discrete**)
3. `int`: [n_steps, minutes, n_ingredients, rating]
    - quantitative mathamatical operations allowed (**quantitative continuous**)
4. `float`: [avg_rating, calories, total_fat sugar, sodium, protein, sat_fat, carbs]
    - quantitative mathamatical operations allowed (**quantitative continuous**)
5. `Timestamp`: [recipe_date, review_date]
    - quantitative mathamatical operations allowed (**quantitative continuous**)

In [329]:
display_df(pd.DataFrame(cleaned.dtypes), 23)

,0
name,object
minutes,int64
contributor_id,object
recipe_date,datetime64[ns]
tags,object
n_steps,int64
steps,object
description,object
ingredients,object
n_ingredients,int64


## Univariate Analysis:
**Analysis**:
Look at the distributions of relevant columns separately by using DataFrame operations and drawing at least two relevant plots.
1. Notice that `rating` is a **discrete quanitative varaibale** while `avg_rating` is a **continuous quantitative variable**

**Report**:
Embed at least one plotly plot you created in your notebook that displays the distribution of a single column (see Part 2: Report for instructions). Include a 1-2 sentence explanation about your plot, making sure to describe and interpret any trends present. (Your notebook will likely have more visualizations than your website, and that’s fine. Feel free to embed more than one univariate visualization in your website if you’d like, but make sure that each embedded plot is accompanied by a description.)


In [279]:
# cleaned_norm = cleaned.pipe(norm)

In [280]:
# px.violin(cleaned_norm, x=['sodium','calories','n_steps'])

In [281]:
# px.violin(cleaned, x='rating')

Score of **5** seems to be the most common score

`Minutes` seems to be a reserve word, differentiate the capitalized version

In [282]:
# px.violin(cleaned, x='n_steps')

In [283]:
# px.violin(cleaned, x='n_ingredients')

## Outlier Filtering:
Obvious outlier exist from looking at the **Univariate Analysis**

In [284]:
# cleaned_filtered = cleaned_norm.pipe(outlier)
# px.violin(cleaned_filtered, x=['n_steps','sodium'])

## Bivariate Analysis:
**Analysis**:
Look at the statistics of pairs of columns to identify possible associations. For instance, you may create scatter plots and plot conditional distributions, or box-plots. You must plot at least two such plots in your notebook. The results of your bivariate analyses will be helpful in identifying interesting hypothesis tests!

**Report**:
Embed at least one plotly plot that displays the relationship between two columns. Include a 1-2 sentence explanation about your plot, making sure to describe and interpret any trends present. (Your notebook will likely have more visualizations than your website, and that’s fine. Feel free to embed more than one bivariate visualization in your website if you’d like, but make sure that each embedded plot is accompanied by a description.)

In [336]:
# Must aggregate first to even see some trend from massive data set
grouped_cleaned = cleaned.pipe(outlier).pipe(group_recipe)
#step3 = step3.assign(sqrt_calories = np.sqrt(step3['calories']))
grouped_cleaned

,minutes,n_steps,n_ingredients,avg_rating,...,review_date,review,recipe_date,tags
recipe_id,,,,,,,,,
275022.0,50.0,11.0,7.0,3.0,...,2008-04-07,Easy comfort food! I definitely thought it was...,2008-01-01,"[60-minutes-or-less, time-to-make, course, mai..."
275026.0,45.0,7.0,9.0,3.0,...,2013-09-21,"Sorry, this one didn&#039;t work out so well. ...",2008-01-01,"[60-minutes-or-less, time-to-make, course, mai..."
275036.0,15.0,6.0,8.0,5.0,...,2008-09-27,This was very nice! I used red bell pepper and...,2008-01-01,"[15-minutes-or-less, time-to-make, course, mai..."
...,...,...,...,...,...,...,...,...,...
537671.0,135.0,54.0,12.0,0.0,...,2018-12-15,"These are an absolute work of art, but just to...",2018-11-28,"[time-to-make, course, preparation, occasion, ..."
537716.0,40.0,12.0,13.0,5.0,...,2018-12-06,These were the best game day sandwiches. I cou...,2018-12-04,"[60-minutes-or-less, time-to-make, course, mai..."
nan,45.0,6.0,5.0,NaN,...,NaT,None,2008-07-21,"[60-minutes-or-less, time-to-make, course, pre..."


In [309]:
fig = px.scatter(grouped_cleaned, x='calories',y='avg_rating',)#trendline="lowess",title="Locally Weighted Smoothing Scatter Plot")
fig.show()

In [310]:
# fig = px.histogram(step4, x='n_steps',y='avg_rating', title='Proportion of Number of Steps')
# fig.show()

In [311]:
fig = px.scatter_matrix(grouped_cleaned, dimensions=["calories", "total_fat",'sugar'], color='rating')
fig.show()

In [312]:
fig = px.scatter(grouped_cleaned, x='calories',y='total_fat',trendline="ols",title="Total Fat/Calories Scatter Plot")
fig.show()

## Interesting Aggregates:
**Analysis**:
Choose columns to group and pivot by and examine aggregate statistics.

**Report**:
Embed at least one grouped table or pivot table in your website and explain its significance.

### Aggregate bu `user_id`

In [313]:
grouped_cleaned.reset_index().set_index('user_id')['steps'].loc['240552.0'][0]

['heat oven to 400 degrees fahrenheit',
 'grease a pie plate 10 x 1 1 / 2 inches',
 'mix 2 cups cheese and the macaroni',
 'sprinkle mixture in the plate',
 'beat remaining ingredients ',
 'except the 1 / 4 cup cheese ',
 'until smooth ',
 '15 seconds in a blender on high ',
 'or 1 minute with a hand beater',
 'pour into plate',
 'bake until knife inserted in center comes out clean - about 40 minutes',
 'sprinkle with 1 / 4 cup cheese',
 'bake until cheese is melted ',
 '1 to 2 minutes',
 'cool 10 minutes',
 'serves 6 to 8 people']

In [314]:
group_user_df = grouped_cleaned.pipe(group_user)

/Users/kevinb/Desktop/dsc80/dsc80_proj/utils/eda.py:102: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [315]:
group_user_df.loc['240552.0']

steps     [heat oven to 400 degrees fahrenheit, grease a...
name      [impossible macaroni and cheese pie, awesome s...
tags      [60-minutes-or-less, time-to-make, course, mai...
rating                                                 4.67
Name: 240552.0, dtype: object

In [316]:
len(group_user_df['name'].loc['240552.0'])

42

### Pivot Table for `id`s and `rating`s

In [317]:
grouped_cleaned.columns

Index(['minutes', 'n_steps', 'n_ingredients', 'avg_rating', 'rating',
       'calories', 'total_fat', 'sugar', 'sodium', 'protein', 'sat_fat',
       'carbs', 'steps', 'name', 'description', 'ingredients', 'user_id',
       'contributor_id', 'review_date', 'review', 'recipe_date', 'tags'],
      dtype='object')

In [347]:
# works just like one hot encoding
grouped_cleaned.pivot_table(
    index='user_id',
    columns='rating',
    values='calories', #this doesn't matter
    aggfunc='count',
    ).fillna(0)

rating,0,1,2,3,4,5
user_id,,,,,,
1000048.0,0.0,0.0,0.0,0.0,0.0,1.0
1000481.0,0.0,0.0,0.0,0.0,1.0,0.0
1000536.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
99979.0,0.0,0.0,0.0,0.0,1.0,0.0
99994.0,0.0,0.0,0.0,0.0,0.0,1.0
nan,1.0,0.0,0.0,0.0,0.0,0.0


In [344]:
grouped_cleaned.pivot_table(
    index='recipe_id',
    columns='rating',
    values='carbs',
    aggfunc='count',
    ).fillna(0)

rating,0,1,2,3,4,5
recipe_id,,,,,,
275022.0,0.0,0.0,0.0,1.0,0.0,0.0
275026.0,0.0,0.0,0.0,1.0,0.0,0.0
275036.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
537671.0,1.0,0.0,0.0,0.0,0.0,0.0
537716.0,0.0,0.0,0.0,0.0,0.0,1.0
nan,1.0,0.0,0.0,0.0,0.0,0.0


In [346]:
grouped_cleaned.pivot_table(
    index='contributor_id',
    columns='rating',
    values='carbs',
    aggfunc='count',
    ).fillna(0)

rating,0,1,2,3,4,5
contributor_id,,,,,,
1000802,0.0,0.0,0.0,0.0,0.0,1.0
100098,0.0,0.0,0.0,0.0,0.0,1.0
1001007,0.0,0.0,0.0,1.0,0.0,2.0
...,...,...,...,...,...,...
999578,0.0,0.0,0.0,0.0,1.0,0.0
999759,1.0,0.0,0.0,2.0,3.0,1.0
999936,0.0,0.0,1.0,1.0,2.0,2.0
